In [1]:
import tensorflow as tf
import math

In [2]:
embedding_size = 5
init_width = 0.5 / embedding_size
emb = tf.Variable(tf.random_uniform(
                    [embedding_size, 2], -init_width, init_width),
                name="emb")
sm_w_t = tf.Variable(
                tf.zeros([embedding_size, 2]),
                name="sm_w_t")
sm_b = tf.Variable(tf.zeros([embedding_size]), name="sm_b")


In [3]:
examples = tf.Variable([1,2])
labels = tf.Variable([3,4])

In [42]:
r1 = np.array([1,2,3])
r2 = np.array([4,5])
np.multiply(r1[:, None], r2[None, :])

array([[ 4,  5],
       [ 8, 10],
       [12, 15]])

In [4]:
# Embeddings for examples: [batch_size, emb_dim]
example_emb = tf.nn.embedding_lookup(emb, examples)
# Weights for labels: [batch_size, emb_dim]
true_w = tf.nn.embedding_lookup(sm_w_t, labels)
# Biases for labels: [batch_size, 1]
true_b = tf.nn.embedding_lookup(sm_b, labels)

In [5]:
num_samples = 3
sampled_ids = tf.Variable([0,1,2])
sampled_w = tf.nn.embedding_lookup(sm_w_t, sampled_ids)
# Biases for sampled ids: [num_sampled, 1]
sampled_b = tf.nn.embedding_lookup(sm_b, sampled_ids)

In [6]:
sampled_b_vec = tf.reshape(sampled_b, [num_samples])
sampled_b_vec = tf.constant([1,2,3], dtype=tf.float32)

In [7]:
sampled_logits = tf.matmul(example_emb,
                                       sampled_w,
                                       transpose_b=True) + sampled_b_vec
true_logits = tf.reduce_sum(tf.multiply(example_emb, true_w), 1) + true_b

In [15]:
labels = tf.Variable([3,4])
labels_matrix = tf.reshape(
                tf.cast(labels,
                        dtype=tf.int64),
                [2, 1])
num_samples = 3
sampled_ids, _, _ = (tf.nn.fixed_unigram_candidate_sampler(
                true_classes=labels_matrix,
                num_true=1,
                num_sampled=num_samples,
                unique=True,
                range_max=5,
                distortion=0.75,
                unigrams=[1.0,1.0,1.0,1.0,1.0]))
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(sampled_ids))

[0 4 1]


In [37]:
def atan2(x, y, epsilon=1.0e-12):
        """
        A hack until the tf developers implement a function that can find the angle from an x and y co-ordinate.
        :param x: 
        :param epsilon: 
        :return: 
        """
        # Add a small number to all zeros, to avoid division by zero:
        x = tf.where(tf.equal(x, 0.0), x+epsilon, x)
        y = tf.where(tf.equal(y, 0.0), y+epsilon, y)
    
        angle = tf.where(tf.greater(x,0.0), tf.atan(y/x), tf.zeros_like(x))
        angle = tf.where(tf.logical_and(tf.less(x,0.0),  tf.greater_equal(y,0.0)), tf.atan(y/x) + np.pi, angle)
        angle = tf.where(tf.logical_and(tf.less(x,0.0),  tf.less(y,0.0)), tf.atan(y/x) - np.pi, angle)
        angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.greater(y,0.0)), 0.5*np.pi * tf.ones_like(x), angle)
        angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.less(y,0.0)), -0.5*np.pi * tf.ones_like(x), angle)
        angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.equal(y,0.0)), tf.zeros_like(x), angle)
        return angle

In [44]:
init_width = 0.25
emb = tf.Variable(tf.random_uniform([5, 2], -init_width, init_width), name="emb")
radius = tf.sqrt(tf.square(emb[:, 0]) + tf.square(emb[:, 1]))
smw = tf.Variable(tf.zeros([5,2]))
theta = atan2(smw[:,1], smw[:, 0])

In [45]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
# print(sess.run(sampled_b))
# print(sess.run(sampled_w))
# print(sess.run(sampled_b_vec))
print(sess.run(emb))
radius = sess.run(radius)
theta = sess.run(theta)
print radius
print theta
# print radius*np.cos(theta)
# print radius*np.sin(theta)


[[ 0.17610693 -0.19265127]
 [ 0.14702195  0.14185423]
 [-0.13774759 -0.12281919]
 [-0.02373785  0.14176118]
 [-0.19075501 -0.21062404]]
[ 0.26101372  0.20429897  0.18455064  0.14373489  0.28416538]
[ 0.78539819  0.78539819  0.78539819  0.78539819  0.78539819]
